In [54]:
#%autoreload
from keras.preprocessing import image
from keras import applications
from keras.models import Sequential
from keras.applications import vgg16
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from keras.layers import Conv2D, Conv3D,Input, ZeroPadding3D, Reshape
from keras.layers.convolutional import Convolution2D, Convolution3D, MaxPooling2D, ZeroPadding2D,ZeroPadding3D 
from keras.layers.core import Reshape
import os
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.callbacks import CSVLogger
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.optimizers import SGD
from keras.layers import Merge 
import numpy as np
import keras
import keras.backend as K
from keras.callbacks import ModelCheckpoint
from keras.callbacks import LambdaCallback
#import imp
#imp.reload(parse_data)

In [6]:
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))
   # return np.linalg.norm(y_pred-y_true)/np.linalg.norm(y_true)

In [31]:
### Camera 1
#rgb
model_left = Sequential()
rgb_model_vgg16 = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(299,299,3))
model_left.add(rgb_model_vgg16)
model_left.get_layer(name='vgg16_input').name='vgg16_left_input'
model_left.get_layer(name='vgg16').name='vgg16_left'
#depth
model_right = Sequential()
depth_model_vgg16 = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(299,299,3))
model_right.add(depth_model_vgg16)
model_right.get_layer(name='vgg16_input').name='vgg16_right_input'
model_right.get_layer(name='vgg16').name='vgg16_right'



#merge
model = Sequential()
model.add(Merge([model_left,model_right], mode='concat'))  
model.add(Flatten())
model.add(Dense(1024, activation='tanh'))
model.add(Dropout(0.5))
#model.add(Dense(1024, activation='relu'))
#model.add(Dense(512, activation='tanh'))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
# model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation = 'relu')) 
print(" DEBUG ")

Adm = keras.optimizers.Adam(lr = 0.0001)
model.compile(loss='mean_squared_error', optimizer=Adm,
            metrics=['mean_squared_error'])
model.summary()






 DEBUG 
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_4 (Merge)              (None, 9, 9, 1024)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 82944)             0         
_________________________________________________________________
dense_11 (Dense)             (None, 1024)              84935680  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 3)                 1539      
To

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:19: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [27]:
object_path = "train/"
dev_path = "dev/"
filename ="vgg"
rescale=1. / 255
img_width, img_height = 299, 299

In [28]:
dev_num = 2
x_dev = np.empty([0,299,299,3])
d_dev = np.empty([0,299,299,3])
y_dev = np.empty([0,3])

for c in range(1,4):
    for k in range(dev_num):
        x_dev = np.load(dev_path+"cameraX.npy")
        d_dev = np.load(dev_path+"cameraD.npy")

        y_dev = np.load(dev_path+"cameraY.npy")

        #print(y_dev)


    
print y_dev.shape, np.min(y_dev),np.max(y_dev)
print x_dev.shape
print y_dev[0]

(252, 3) -1.4735804796218872 3.609727144241333
(252, 299, 299, 3)
[0.36287004 0.63130355 2.23632264]


In [57]:
def train(model):
    filename = "vgg2d_"
    fh = open(filename + 'report.txt','w')
    csv_logger = CSVLogger('logvgg.csv', append=True, separator=';') 
    #f_train = open(filename + 'train_report.txt',"a", 0)
    #f_train_step = open(filename + 'step_report.txt',"a", 0)
    #f_dev = open(filename + 'dev_report.txt',"a", 0)
    batch = 32
    epochs = 30
    batch_num = 8
    #batch_num = 60
    rescale=1. / 255
    filepath_best="weights.best.hdf5"
    #if os.path.exists(filepath_best):
        #model.load_weights(filepath_best)
        #print "load weight success!"

    X = np.load(object_path+"cameraX.npy")
    D = np.load(object_path+"cameraD.npy")
    Y = np.load(object_path+"cameraY.npy")

    print len(X)
    print len(D)
    print len(Y)
    
    def batchOutput(batch, logs):
        print("Finished batch: " + str(batch))
        print(logs)

    batchLogCallback = LambdaCallback(on_batch_end=batchOutput)
    


    checkpoint = ModelCheckpoint(filepath=filepath_best,monitor='loss', verbose=1,save_best_only=True, mode='min')
    res = model.fit([X,D], Y ,batch_size=32, epochs=3,verbose=2, shuffle=True,validation_data=([x_dev, d_dev], y_dev),callbacks=[batchLogCallback])
    #res = model.fit([X,D], Y ,batch_size=32, epochs=3,verbose=1, shuffle=True,validation_split = 0.2)
    print("############HISTORY###############")
    print(str(res.history))
    #checkpoint = ModelCheckpoint(filepath=filepath_best,monitor='loss', verbose=1,save_best_only=True, mode='min')    
    #train_loss = model.evaluate([x_batch, d_batch], y = y_batch)
    #print "train_loss",train_loss
    #f_train.write(str(train_loss))
    dev_loss = model.evaluate(x = [x_dev, d_dev], y = y_dev)
    #f_dev.write(str(dev_loss))
    print "----val_loss & l2_error----:"
    print dev_loss
    #f_train.close()
    #f_train_step.close()
    #f_dev.close()

In [ ]:
train(model)


1052
1052
1052
Train on 1052 samples, validate on 252 samples
Epoch 1/3
Finished batch: 0
{'mean_squared_error': 0.9595883, 'loss': 0.9595883, 'batch': 0, 'size': 32}
Finished batch: 1
{'mean_squared_error': 1.5978246, 'loss': 1.5978246, 'batch': 1, 'size': 32}
Finished batch: 2
{'mean_squared_error': 1.620384, 'loss': 1.620384, 'batch': 2, 'size': 32}
Finished batch: 3
{'mean_squared_error': 1.335381, 'loss': 1.335381, 'batch': 3, 'size': 32}
Finished batch: 4
{'mean_squared_error': 1.7382153, 'loss': 1.7382153, 'batch': 4, 'size': 32}
Finished batch: 5
{'mean_squared_error': 1.2429922, 'loss': 1.2429922, 'batch': 5, 'size': 32}
Finished batch: 6
{'mean_squared_error': 1.6344616, 'loss': 1.6344616, 'batch': 6, 'size': 32}
Finished batch: 7
{'mean_squared_error': 1.5115302, 'loss': 1.5115302, 'batch': 7, 'size': 32}
Finished batch: 8
{'mean_squared_error': 1.3870099, 'loss': 1.3870099, 'batch': 8, 'size': 32}
Finished batch: 9
{'mean_squared_error': 1.4860326, 'loss': 1.4860326, 'batch